# Pre-built Hamiltonians and Observables in NetKet

This notebook explores NetKet's extensive library of pre-built Hamiltonians and observables, which are essential for quantum many-body simulations. We'll learn how to construct common quantum models and measure physical quantities.

## Learning Objectives:
- Understand NetKet's pre-built Hamiltonian library
- Learn to construct common quantum models (Ising, Heisenberg, Hubbard)
- Explore observables and measurement operators
- Practice combining Hamiltonians for complex models
- Understand local vs global observables

In [ ]:
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
from netket.graph import Hypercube, Chain, Square
from netket.hilbert import Spin, Fock
from netket.operator import spin, boson
import warnings
warnings.filterwarnings('ignore')

print(f"NetKet version: {nk.__version__}")
print(f"JAX version: {jax.__version__}")

## 1. Spin Hamiltonians

NetKet provides several pre-built spin Hamiltonians that are commonly used in condensed matter physics.

### 1.1 Transverse Field Ising Model (TFIM)

The TFIM is one of the most studied quantum models:
$$H = -J \sum_{\langle i,j \rangle} \sigma^z_i \sigma^z_j - h \sum_i \sigma^x_i$$

In [ ]:
# Create a 1D chain for TFIM
L = 10
graph = Chain(L, pbc=True)  # periodic boundary conditions
hilbert = Spin(s=1/2, N=L)

# Parameters
J = 1.0  # coupling strength
h = 0.5  # transverse field strength

# Method 1: Using pre-built TFIM
tfim = nk.operator.Ising(hilbert=hilbert, graph=graph, J=J, h=h)

print(f"TFIM Hamiltonian matrix shape: {tfim.to_dense().shape}")
print(f"Hilbert space dimension: {hilbert.n_states}")

# Method 2: Manual construction using Pauli operators
tfim_manual = 0.0
# ZZ interactions
for edge in graph.edges():
    tfim_manual += -J * nk.operator.spin.sigmaz(hilbert, edge[0]) * nk.operator.spin.sigmaz(hilbert, edge[1])

# X field
for i in range(L):
    tfim_manual += -h * nk.operator.spin.sigmax(hilbert, i)

print(f"Manual TFIM equals pre-built: {np.allclose(tfim.to_dense(), tfim_manual.to_dense())}")

### 1.2 Heisenberg Model

The Heisenberg model includes interactions in all three spatial directions:
$$H = J \sum_{\langle i,j \rangle} (\sigma^x_i \sigma^x_j + \sigma^y_i \sigma^y_j + \sigma^z_i \sigma^z_j)$$

In [ ]:
# XXZ Heisenberg model (anisotropic)
Jx, Jy, Jz = 1.0, 1.0, 0.5  # anisotropic couplings
h = 0.1  # magnetic field

# Using pre-built Heisenberg
heisenberg = nk.operator.Heisenberg(hilbert=hilbert, graph=graph, 
                                  J=[Jx, Jy, Jz], h=h)

# Isotropic Heisenberg (J=[1,1,1])
heisenberg_iso = nk.operator.Heisenberg(hilbert=hilbert, graph=graph)

print(f"Anisotropic Heisenberg shape: {heisenberg.to_dense().shape}")
print(f"Ground state energy (small system): {np.min(np.real(np.linalg.eigvals(heisenberg.to_dense()))):.4f}")

# Manual construction for comparison
heisenberg_manual = 0.0
for edge in graph.edges():
    i, j = edge
    heisenberg_manual += Jx * nk.operator.spin.sigmax(hilbert, i) * nk.operator.spin.sigmax(hilbert, j)
    heisenberg_manual += Jy * nk.operator.spin.sigmay(hilbert, i) * nk.operator.spin.sigmay(hilbert, j)
    heisenberg_manual += Jz * nk.operator.spin.sigmaz(hilbert, i) * nk.operator.spin.sigmaz(hilbert, j)

for i in range(L):
    heisenberg_manual += h * nk.operator.spin.sigmaz(hilbert, i)

print(f"Manual Heisenberg equals pre-built: {np.allclose(heisenberg.to_dense(), heisenberg_manual.to_dense())}")

### 1.3 Spin-1 Models

NetKet also supports higher spin systems.

In [ ]:
# Spin-1 system
L_s1 = 6  # smaller system for spin-1
hilbert_s1 = Spin(s=1, N=L_s1)
graph_s1 = Chain(L_s1, pbc=True)

# Spin-1 AKLT model (a famous exactly solvable model)
# H = sum_i [S_i · S_{i+1} + (1/3)(S_i · S_{i+1})^2]
aklt = 0.0
for i in range(L_s1):
    j = (i + 1) % L_s1
    # S_i · S_{i+1} term
    dot_product = (nk.operator.spin.sigmax(hilbert_s1, i) * nk.operator.spin.sigmax(hilbert_s1, j) +
                   nk.operator.spin.sigmay(hilbert_s1, i) * nk.operator.spin.sigmay(hilbert_s1, j) +
                   nk.operator.spin.sigmaz(hilbert_s1, i) * nk.operator.spin.sigmaz(hilbert_s1, j))
    
    aklt += dot_product + (1/3) * dot_product * dot_product

print(f"Spin-1 Hilbert space dimension: {hilbert_s1.n_states}")
print(f"AKLT Hamiltonian constructed successfully")

# Alternative: using Heisenberg with biquadratic term
heisenberg_s1 = nk.operator.Heisenberg(hilbert=hilbert_s1, graph=graph_s1)
print(f"Spin-1 Heisenberg energy scale: {np.min(np.real(np.linalg.eigvals(heisenberg_s1.to_dense()))):.4f}")

## 2. Fermionic Hamiltonians

For fermionic systems, NetKet provides tools for constructing Hubbard-type models.

### 2.1 Hubbard Model

The Hubbard model is fundamental for understanding strongly correlated electrons:
$$H = -t \sum_{\langle i,j \rangle, \sigma} (c^\dagger_{i\sigma} c_{j\sigma} + h.c.) + U \sum_i n_{i\uparrow} n_{i\downarrow}$$

In [ ]:
# Small Hubbard model (2x2 lattice)
L_hub = 4
graph_hub = Chain(L_hub, pbc=True)

# Fermionic Hilbert space (spin-1/2 fermions)
hilbert_hub = nk.hilbert.Fock(n_max=1, N=2*L_hub)  # 2*L for spin up and down

# Hubbard model parameters
t = 1.0  # hopping
U = 4.0  # on-site interaction

# Pre-built Hubbard model
hubbard = nk.operator.Hubbard(hilbert=hilbert_hub, graph=graph_hub, U=U, t=t)

print(f"Hubbard model constructed")
print(f"Hilbert space size: {hilbert_hub.n_states}")

# Note: For larger systems, we typically use variational methods rather than exact diagonalization

### 2.2 Extended Hubbard Model

We can add nearest-neighbor density interactions.

In [ ]:
# Extended Hubbard with nearest-neighbor interaction
V = 0.5  # nearest-neighbor interaction

# This requires manual construction as it's not a standard pre-built model
# For demonstration, we'll show the concept
print(f"Extended Hubbard would add V*n_i*n_j terms between neighbors")
print(f"Parameters: t={t}, U={U}, V={V}")

## 3. Bosonic Hamiltonians

NetKet supports bosonic systems through the Fock space representation.

### 3.1 Bose-Hubbard Model

The Bose-Hubbard model describes interacting bosons on a lattice:
$$H = -t \sum_{\langle i,j \rangle} (b^\dagger_i b_j + h.c.) + \frac{U}{2} \sum_i n_i(n_i-1) - \mu \sum_i n_i$$

In [ ]:
# Bose-Hubbard model
L_bh = 4
n_max = 3  # maximum occupation per site
graph_bh = Chain(L_bh, pbc=True)
hilbert_bh = Fock(n_max=n_max, N=L_bh)

# Parameters
t_bh = 1.0   # hopping
U_bh = 5.0   # on-site interaction
mu = 2.0     # chemical potential

# Pre-built Bose-Hubbard
bose_hubbard = nk.operator.BoseHubbard(hilbert=hilbert_bh, graph=graph_bh, 
                                      U=U_bh, t=t_bh, mu=mu)

print(f"Bose-Hubbard model constructed")
print(f"Hilbert space size: {hilbert_bh.n_states}")
print(f"Max occupation per site: {n_max}")

# Manual construction for understanding
bh_manual = 0.0

# Hopping terms
for edge in graph_bh.edges():
    i, j = edge
    bh_manual += -t_bh * (nk.operator.boson.create(hilbert_bh, i) * nk.operator.boson.destroy(hilbert_bh, j) +
                          nk.operator.boson.create(hilbert_bh, j) * nk.operator.boson.destroy(hilbert_bh, i))

# On-site interaction and chemical potential
for i in range(L_bh):
    n_i = nk.operator.boson.number(hilbert_bh, i)
    bh_manual += (U_bh/2) * n_i * (n_i - 1) - mu * n_i

print(f"Manual BH equals pre-built: {np.allclose(bose_hubbard.to_dense(), bh_manual.to_dense())}")

## 4. Observables and Measurements

Observables are operators whose expectation values we want to measure.

### 4.1 Local Observables

Local observables act on individual sites or small clusters.

In [ ]:
# Back to spin system for observables
L_obs = 8
graph_obs = Chain(L_obs, pbc=True)
hilbert_obs = Spin(s=1/2, N=L_obs)

# Local spin observables
sigma_x_0 = nk.operator.spin.sigmax(hilbert_obs, 0)  # σ^x at site 0
sigma_z_0 = nk.operator.spin.sigmaz(hilbert_obs, 0)  # σ^z at site 0

# Magnetization in different directions
mag_x = sum(nk.operator.spin.sigmax(hilbert_obs, i) for i in range(L_obs)) / L_obs
mag_z = sum(nk.operator.spin.sigmaz(hilbert_obs, i) for i in range(L_obs)) / L_obs

print(f"Local σ^x operator at site 0 shape: {sigma_x_0.to_dense().shape}")
print(f"Magnetization operators constructed")

# Two-point correlation functions
sigma_z_corr = nk.operator.spin.sigmaz(hilbert_obs, 0) * nk.operator.spin.sigmaz(hilbert_obs, L_obs//2)

print(f"Two-point correlation operator: σ^z_0 σ^z_{L_obs//2}")

### 4.2 Global Observables

Global observables depend on the entire system state.

In [ ]:
# Total spin components
S_x_total = sum(nk.operator.spin.sigmax(hilbert_obs, i) for i in range(L_obs))
S_y_total = sum(nk.operator.spin.sigmay(hilbert_obs, i) for i in range(L_obs))
S_z_total = sum(nk.operator.spin.sigmaz(hilbert_obs, i) for i in range(L_obs))

# Total spin squared: S^2 = S_x^2 + S_y^2 + S_z^2
S_squared = S_x_total*S_x_total + S_y_total*S_y_total + S_z_total*S_z_total

print(f"Total spin operators constructed")
print(f"S^2 operator has {S_squared.n_conn_max} maximum connections")

# Energy density (Hamiltonian per site)
tfim_obs = nk.operator.Ising(hilbert=hilbert_obs, graph=graph_obs, J=1.0, h=0.5)
energy_density = tfim_obs / L_obs

print(f"Energy density operator constructed")

### 4.3 Structure Factors

Structure factors are important for understanding spatial correlations.

In [ ]:
def structure_factor_z(hilbert, k):
    """Compute S^z structure factor at momentum k"""
    L = hilbert.size
    S_k = 0.0
    
    for i in range(L):
        for j in range(L):
            phase = np.exp(1j * k * (i - j))
            S_k += phase * nk.operator.spin.sigmaz(hilbert, i) * nk.operator.spin.sigmaz(hilbert, j)
    
    return S_k / L

# Structure factor at k=π (antiferromagnetic)
S_pi = structure_factor_z(hilbert_obs, np.pi)
print(f"S^z(π) structure factor constructed")

# Uniform susceptibility (k=0)
S_0 = structure_factor_z(hilbert_obs, 0.0)
print(f"S^z(0) uniform susceptibility constructed")

## 5. Composite and Custom Hamiltonians

Often we need to combine multiple terms or create custom Hamiltonians.

### 5.1 Combining Hamiltonians

NetKet allows easy combination of different Hamiltonian terms.

In [ ]:
# Complex model: TFIM + longitudinal field + NNN interactions
L_comp = 6
graph_comp = Chain(L_comp, pbc=True)
hilbert_comp = Spin(s=1/2, N=L_comp)

# Main TFIM
h_tfim = nk.operator.Ising(hilbert=hilbert_comp, graph=graph_comp, J=1.0, h=0.8)

# Longitudinal field
h_long = sum(nk.operator.spin.sigmaz(hilbert_comp, i) for i in range(L_comp))

# Next-nearest neighbor interactions
h_nnn = 0.0
for i in range(L_comp):
    j = (i + 2) % L_comp  # next-nearest neighbor
    h_nnn += 0.3 * nk.operator.spin.sigmaz(hilbert_comp, i) * nk.operator.spin.sigmaz(hilbert_comp, j)

# Combined Hamiltonian
h_total = h_tfim + 0.2 * h_long + h_nnn

print(f"Combined Hamiltonian constructed")
print(f"Total terms: TFIM + longitudinal field + NNN interactions")

# Check that it's Hermitian
h_dense = h_total.to_dense()
is_hermitian = np.allclose(h_dense, h_dense.conj().T)
print(f"Hamiltonian is Hermitian: {is_hermitian}")

### 5.2 Custom Interaction Patterns

We can create Hamiltonians with arbitrary interaction patterns.

In [ ]:
# Custom interaction graph: star configuration
from netket.graph import Graph

# Create a star graph (central site connected to all others)
def create_star_graph(n_sites):
    edges = []
    for i in range(1, n_sites):
        edges.append([0, i])  # center (0) connected to all others
    return Graph(edges=edges)

star_graph = create_star_graph(5)
hilbert_star = Spin(s=1/2, N=5)

# Ising model on star graph
h_star = nk.operator.Ising(hilbert=hilbert_star, graph=star_graph, J=1.0, h=0.3)

print(f"Star graph Ising model constructed")
print(f"Number of edges: {len(star_graph.edges())}")
print(f"Edges: {list(star_graph.edges())}")

# All-to-all interaction (mean-field like)
h_all_to_all = 0.0
J_all = 0.1  # weak all-to-all coupling

for i in range(5):
    for j in range(i+1, 5):
        h_all_to_all += J_all * nk.operator.spin.sigmaz(hilbert_star, i) * nk.operator.spin.sigmaz(hilbert_star, j)

print(f"All-to-all interaction terms: {5*4//2}")

## 6. Advanced Observable Techniques

Let's explore more sophisticated observable calculations.

### 6.1 Dynamical Correlation Functions

These require time evolution operators (advanced topic).

In [ ]:
# Spectral function building blocks
# For actual calculations, we'd need time evolution

# Current operator for optical conductivity
def current_operator(hilbert, graph, direction='x'):
    """Create current operator for charge transport"""
    j_op = 0.0
    
    for edge in graph.edges():
        i, j = edge
        # Simplified current operator (exact form depends on model)
        if direction == 'x':
            j_op += 1j * (nk.operator.spin.sigmax(hilbert, i) * nk.operator.spin.sigmay(hilbert, j) -
                         nk.operator.spin.sigmay(hilbert, i) * nk.operator.spin.sigmax(hilbert, j))
    
    return j_op

j_x = current_operator(hilbert_obs, graph_obs, 'x')
print(f"Current operator constructed for transport calculations")

### 6.2 Entanglement Measures

NetKet can compute various entanglement measures.

In [ ]:
# For small systems, we can compute entanglement entropy
def bipartite_entanglement_entropy(state, subsystem_sites):
    """
    Compute entanglement entropy for a bipartition
    Note: This is a simplified version for demonstration
    """
    # This would require reduced density matrix calculations
    # which are available in NetKet for small systems
    print(f"Entanglement entropy calculation would involve sites: {subsystem_sites}")
    return "Entanglement calculation framework"

# Example bipartition
left_sites = list(range(L_obs//2))
ee_framework = bipartite_entanglement_entropy(None, left_sites)
print(f"Left subsystem sites: {left_sites}")
print(f"Right subsystem sites: {list(range(L_obs//2, L_obs))}")

## 7. Practical Example: Phase Transition Study

Let's combine everything to study a quantum phase transition.

In [ ]:
# Study TFIM phase transition
def study_tfim_transition():
    L = 8
    graph = Chain(L, pbc=True)
    hilbert = Spin(s=1/2, N=L)
    
    # Range of transverse fields
    h_values = np.linspace(0.1, 2.0, 10)
    
    observables = {
        'energy': [],
        'magnetization_x': [],
        'magnetization_z': [],
        'susceptibility': []
    }
    
    for h in h_values:
        # Construct Hamiltonian
        H = nk.operator.Ising(hilbert=hilbert, graph=graph, J=1.0, h=h)
        
        # Observables
        mag_x = sum(nk.operator.spin.sigmax(hilbert, i) for i in range(L)) / L
        mag_z = sum(nk.operator.spin.sigmaz(hilbert, i) for i in range(L)) / L
        
        # For exact calculation (small system)
        eigenvals, eigenvecs = np.linalg.eigh(H.to_dense())
        gs_energy = eigenvals[0]
        gs_state = eigenvecs[:, 0]
        
        # Expectation values
        mx_exp = np.real(gs_state.conj() @ mag_x.to_dense() @ gs_state)
        mz_exp = np.real(gs_state.conj() @ mag_z.to_dense() @ gs_state)
        
        # Susceptibility (second derivative of energy)
        chi = np.real(gs_state.conj() @ (mag_x @ mag_x).to_dense() @ gs_state) - mx_exp**2
        
        observables['energy'].append(gs_energy/L)
        observables['magnetization_x'].append(abs(mx_exp))
        observables['magnetization_z'].append(abs(mz_exp))
        observables['susceptibility'].append(chi)
    
    return h_values, observables

h_vals, obs = study_tfim_transition()

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0,0].plot(h_vals, obs['energy'], 'bo-')
axes[0,0].set_xlabel('Transverse field h')
axes[0,0].set_ylabel('Ground state energy per site')
axes[0,0].set_title('Energy vs Field')

axes[0,1].plot(h_vals, obs['magnetization_x'], 'ro-', label='⟨σ^x⟩')
axes[0,1].plot(h_vals, obs['magnetization_z'], 'go-', label='⟨σ^z⟩')
axes[0,1].set_xlabel('Transverse field h')
axes[0,1].set_ylabel('Magnetization')
axes[0,1].set_title('Magnetization vs Field')
axes[0,1].legend()

axes[1,0].plot(h_vals, obs['susceptibility'], 'mo-')
axes[1,0].set_xlabel('Transverse field h')
axes[1,0].set_ylabel('Susceptibility')
axes[1,0].set_title('Susceptibility vs Field')

# Energy gap (difference between first two eigenvalues)
gaps = []
for h in h_vals:
    H = nk.operator.Ising(hilbert=Spin(s=1/2, N=8), graph=Chain(8, pbc=True), J=1.0, h=h)
    eigenvals = np.linalg.eigvals(H.to_dense())
    eigenvals = np.sort(np.real(eigenvals))
    gaps.append(eigenvals[1] - eigenvals[0])

axes[1,1].plot(h_vals, gaps, 'co-')
axes[1,1].set_xlabel('Transverse field h')
axes[1,1].set_ylabel('Energy gap')
axes[1,1].set_title('Gap vs Field')

plt.tight_layout()
plt.show()

print(f"Phase transition study completed for L=8 TFIM")
print(f"Critical point around h_c ≈ 1.0 (exact: h_c = 1.0)")

## 8. Performance Considerations

When working with larger systems, several considerations become important.

In [ ]:
# Sparse vs dense representations
L_perf = 12
graph_perf = Chain(L_perf, pbc=True)
hilbert_perf = Spin(s=1/2, N=L_perf)

# Large Hamiltonian
H_large = nk.operator.Ising(hilbert=hilbert_perf, graph=graph_perf, J=1.0, h=0.5)

print(f"Large system: L={L_perf}")
print(f"Hilbert space dimension: {hilbert_perf.n_states}")
print(f"Matrix would be {hilbert_perf.n_states}x{hilbert_perf.n_states}")
print(f"Memory for dense matrix: {hilbert_perf.n_states**2 * 16 / 1e9:.2f} GB (complex double)")

# NetKet uses sparse representations and matrix-free methods
print(f"\nNetKet advantages:")
print(f"- Sparse matrix representation")
print(f"- Matrix-free operations")
print(f"- Efficient local operator application")
print(f"- JAX acceleration")

# Connection number (sparsity measure)
print(f"Maximum connections per configuration: {H_large.n_conn_max}")
print(f"This indicates high sparsity suitable for large-scale calculations")

## 9. Exercise Problems

Try these exercises to practice working with Hamiltonians and observables.

### Exercise 1: Custom Spin Model

Create a spin model with:
- Nearest-neighbor Ising interactions (J=1)
- Next-nearest-neighbor interactions (J'=0.3)  
- Transverse field (h=0.5)
- Longitudinal field (h_z=0.2)

Compare the ground state energy with the standard TFIM.

In [ ]:
# Exercise 1 Solution
def exercise_1():
    L = 6
    graph = Chain(L, pbc=True)
    hilbert = Spin(s=1/2, N=L)
    
    # Standard TFIM
    H_tfim = nk.operator.Ising(hilbert=hilbert, graph=graph, J=1.0, h=0.5)
    
    # Custom model
    H_custom = nk.operator.Ising(hilbert=hilbert, graph=graph, J=1.0, h=0.5)
    
    # Add NNN interactions
    for i in range(L):
        j = (i + 2) % L
        H_custom += 0.3 * nk.operator.spin.sigmaz(hilbert, i) * nk.operator.spin.sigmaz(hilbert, j)
    
    # Add longitudinal field
    for i in range(L):
        H_custom += 0.2 * nk.operator.spin.sigmaz(hilbert, i)
    
    # Ground state energies
    E_tfim = np.min(np.real(np.linalg.eigvals(H_tfim.to_dense())))
    E_custom = np.min(np.real(np.linalg.eigvals(H_custom.to_dense())))
    
    print(f"Exercise 1 Results:")
    print(f"TFIM ground state energy: {E_tfim:.4f}")
    print(f"Custom model ground state energy: {E_custom:.4f}")
    print(f"Energy difference: {E_custom - E_tfim:.4f}")
    
    return E_tfim, E_custom

exercise_1()

### Exercise 2: Observable Correlations

Calculate the correlation function ⟨σ^z_0 σ^z_r⟩ for different distances r in the TFIM ground state.

In [ ]:
# Exercise 2 Solution
def exercise_2():
    L = 8
    graph = Chain(L, pbc=True)
    hilbert = Spin(s=1/2, N=L)
    
    # TFIM at critical point
    H = nk.operator.Ising(hilbert=hilbert, graph=graph, J=1.0, h=1.0)
    
    # Ground state
    eigenvals, eigenvecs = np.linalg.eigh(H.to_dense())
    gs_state = eigenvecs[:, 0]
    
    # Correlation function
    correlations = []
    distances = list(range(L//2 + 1))
    
    for r in distances:
        if r == 0:
            # ⟨σ^z_0 σ^z_0⟩ = ⟨σ^z_0⟩^2 (for normalized operators)
            sigma_z_0 = nk.operator.spin.sigmaz(hilbert, 0)
            corr = np.real(gs_state.conj() @ sigma_z_0.to_dense() @ gs_state)**2
        else:
            # ⟨σ^z_0 σ^z_r⟩
            corr_op = nk.operator.spin.sigmaz(hilbert, 0) * nk.operator.spin.sigmaz(hilbert, r)
            corr = np.real(gs_state.conj() @ corr_op.to_dense() @ gs_state)
        
        correlations.append(corr)
    
    print(f"Exercise 2 Results:")
    print(f"Correlation function ⟨σ^z_0 σ^z_r⟩ at h=1.0 (critical point):")
    for r, corr in zip(distances, correlations):
        print(f"r={r}: {corr:.4f}")
    
    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(distances, correlations, 'ro-')
    plt.xlabel('Distance r')
    plt.ylabel('⟨σ^z_0 σ^z_r⟩')
    plt.title('Spin-spin correlation function')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return distances, correlations

exercise_2()

### Exercise 3: Finite-Size Scaling

Study how the energy gap scales with system size in the TFIM.

In [ ]:
# Exercise 3 Solution
def exercise_3():
    sizes = [4, 6, 8, 10, 12]
    gaps = []
    
    for L in sizes:
        if L <= 12:  # Only for manageable sizes
            graph = Chain(L, pbc=True)
            hilbert = Spin(s=1/2, N=L)
            
            # Critical TFIM
            H = nk.operator.Ising(hilbert=hilbert, graph=graph, J=1.0, h=1.0)
            
            # Two lowest eigenvalues
            eigenvals = np.linalg.eigvals(H.to_dense())
            eigenvals = np.sort(np.real(eigenvals))
            gap = eigenvals[1] - eigenvals[0]
            gaps.append(gap)
        else:
            gaps.append(np.nan)  # Too large for exact diagonalization
    
    print(f"Exercise 3 Results:")
    print(f"Energy gaps at critical point h=1.0:")
    for L, gap in zip(sizes, gaps):
        if not np.isnan(gap):
            print(f"L={L}: Δ={gap:.4f}")
        else:
            print(f"L={L}: Too large for exact calculation")
    
    # Plot finite-size scaling
    valid_sizes = [L for L, gap in zip(sizes, gaps) if not np.isnan(gap)]
    valid_gaps = [gap for gap in gaps if not np.isnan(gap)]
    
    if len(valid_gaps) > 2:
        plt.figure(figsize=(8, 5))
        plt.loglog(valid_sizes, valid_gaps, 'bo-')
        plt.xlabel('System size L')
        plt.ylabel('Energy gap Δ')
        plt.title('Finite-size scaling of energy gap')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Fit scaling law Δ ∝ L^(-z)
        log_L = np.log(valid_sizes)
        log_gap = np.log(valid_gaps)
        z = -np.polyfit(log_L, log_gap, 1)[0]
        print(f"Scaling exponent z ≈ {z:.2f}")
        print(f"Gap scales as Δ ∝ L^(-{z:.2f})")
    
    return sizes, gaps

exercise_3()

## Summary

In this notebook, we've covered:

### Key Concepts:
1. **Pre-built Hamiltonians**: TFIM, Heisenberg, Hubbard, Bose-Hubbard
2. **Observables**: Local and global operators, correlation functions
3. **Custom Models**: Combining terms, arbitrary interactions
4. **Practical Applications**: Phase transitions, finite-size scaling

### Important NetKet Features:
- Extensive library of quantum models
- Efficient sparse representations
- Easy combination of Hamiltonian terms
- Support for different particle types (spins, fermions, bosons)
- Integration with JAX for performance

### Next Steps:
- Learn about variational Monte Carlo methods
- Explore ground state optimization
- Study excited states and dynamics
- Apply to realistic materials problems

The pre-built Hamiltonians and observables in NetKet provide a powerful foundation for quantum many-body calculations, from exact studies of small systems to variational approaches for large-scale problems.